In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [1]:
alpaca_prompt = """Below is a question about women health Write a response that appropriately answers the question.

### Question:
{}

### Answer:
{}
"""

In [4]:
import zipfile
with zipfile.ZipFile('lora_model.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [2]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model/content/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "How can women support eye health?", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
x = tokenizer.batch_decode(outputs)



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [6]:
!pip install flask
!pip install pyngrok

In [3]:
from google.colab import userdata

In [97]:
import getpass
import os
import threading


from pyngrok import ngrok, conf
import socket
from flask import Flask, request, render_template, jsonify, redirect
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = userdata.get('NGROK')

app = Flask(__name__)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


app = Flask(__name__)


@app.route("/")
def index():
    return render_template('chat.html')


@app.route("/get", methods=["GET", "POST"])
def chat():
    msg = request.form["msg"]
    input = msg
    return get_Chat_response(input)


def get_Chat_response(text):

    # Let's chat for 5 lines
    inputs = tokenizer(
            [
                alpaca_prompt.format(
                    text, # instruction
                    "" # output - leave this blank for generation!
                )
            ], return_tensors="pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    x = tokenizer.batch_decode(outputs)
    x = x[0]
    ind = x.find("Answer")
    end = x.find('</s>')
    output = x[ind + 9:end]  # Adjust the slicing to fit your output format

    return output



if __name__ == '__main__':
    threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
 * ngrok tunnel "https://0e62-34-133-74-141.ngrok-free.app" -> "http://127.0.0.1:5000/"
